In [1]:
import pandas as pd

In [13]:
full_dataset = pd.read_csv('sent_tokenized_dataset.csv').drop('index', axis=1)

honeypot_paper_ids = [421969, 593236, 422071]
honeypots = full_dataset[full_dataset['paper_id'].isin(honeypot_paper_ids)]
full_dataset = full_dataset[~full_dataset['paper_id'].isin(honeypot_paper_ids)]

testing_paper_ids = full_dataset['paper_id'].unique()[:5]
testing_dataset = full_dataset[full_dataset['paper_id'].isin(testing_paper_ids)]
full_dataset = full_dataset[~full_dataset['paper_id'].isin(testing_paper_ids)]

In [14]:
honeypots

,domain,date,text,paper_id,sentence_id
0,www.ekathimerini.com,2018-07-25 00:00:00,"The leaders of Greece, Bulgaria, Romania and S...",421969,0
1,www.ekathimerini.com,2018-07-25 00:00:00,The prospects of the Western Balkans joining t...,421969,1
2,www.ekathimerini.com,2018-07-25 00:00:00,"Prime ministers Alexis Tsipras of Greece, Boyk...",421969,2
3,www.ekathimerini.com,2018-07-25 00:00:00,The refugee crisis is also expected to be a to...,421969,3
4,www.ekathimerini.com,2018-07-25 00:00:00,The Thessaloniki meeting is seen as an effort ...,421969,4
...,...,...,...,...,...
84,www.ekathimerini.com,2018-07-21 00:00:00,Once-neglected family homes are now holiday ma...,422071,84
85,www.ekathimerini.com,2018-07-21 00:00:00,Maybe they will retire here as well one day.,422071,85
86,www.ekathimerini.com,2018-07-21 00:00:00,"In summer, the square is a hive of activity af...",422071,86
87,www.ekathimerini.com,2018-07-21 00:00:00,"Like their spiritual ancestor Ulysses, these o...",422071,87


In [15]:
testing_dataset

,domain,date,text,paper_id,sentence_id
89,www.news18.com,2018-07-20 00:00:00,"China will ""absolutely not"" fire the first sho...",200071,89
90,www.news18.com,2018-07-20 00:00:00,"Given the 12-hour time difference, that would ...",200071,90
91,www.news18.com,2018-07-20 00:00:00,Other media carried similar reports.But the mi...,200071,91
92,www.news18.com,2018-07-20 00:00:00,"""The Chinese government's position has been st...",200071,92
93,www.news18.com,2018-07-20 00:00:00,"We absolutely will not fire the first shot, an...",200071,93
...,...,...,...,...,...
241,www.americanthinker.com,2018-07-20 00:00:00,"Wants to produce more oil, instead of a presid...",450575,241
242,www.americanthinker.com,2018-07-20 00:00:00,I am sure people can think of many more things...,450575,242
243,www.americanthinker.com,2018-07-20 00:00:00,"Finally, I am extremely thankful and proud tha...",450575,243
244,www.americanthinker.com,2018-07-20 00:00:00,They understood that if only the king and the ...,450575,244


In [63]:
def make_toloka_pool(data, path, scores=None):
    pool = pd.DataFrame(columns=['INPUT:text', 'GOLDEN:is_subjective'])
    for index, row in data.iterrows():
        paper_id = row['paper_id']
        sentence_id = row['sentence_id']

        prev_context = data.loc[max(0, index - 2):index - 1]
        next_context = data.loc[index + 1:min(len(data), index + 4)]

        prev_context = ' '.join(prev_context[prev_context['paper_id'] == paper_id]['text'])
        next_context = ' '.join(next_context[next_context['paper_id'] == paper_id]['text'])

        text= f"{prev_context}{' ' if prev_context else ''}**{row['text']}**{' ' if next_context else ''}{next_context}"
        score = None if scores is None else scores[scores['sentence_id'] == sentence_id]['score'].item()

        pool = pd.concat([pool, pd.DataFrame([{'INPUT:text': text, 'GOLDEN:is_subjective': score}])], ignore_index=True)
    if scores is None:
        pool = pool.drop('GOLDEN:is_subjective', axis=1)
    pool.to_csv(path, sep='\t', index=False)

In [64]:
make_toloka_pool(honeypots, "honeypots.tsv", pd.read_csv("sample_scores.csv"))

In [65]:
make_toloka_pool(testing_dataset, "testing_dataset.tsv")

In [67]:
make_toloka_pool(full_dataset, "full_dataset.tsv")